In [ ]:
import pandas as pd
import json
import os
import glob

# Folder containing all CSVs
folder_path = './input_data'

# Find all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# List to hold each DataFrame
df_list = []

# Loop through each file, read, and tag with city name
for file in csv_files:
    # Extract city name from filename, e.g., 'bangalore' from 'bangalore_cars_parsed.csv'
    city = os.path.basename(file).split('_')[0].capitalize()  # Capitalize for uniformity
    df = pd.read_excel(file)
    df['City'] = city  # Add a column for city
    df_list.append(df)

# Combine all DataFrames into one
df_combined = pd.concat(df_list, ignore_index=True)

# Preview the combined DataFrame
print(df_combined[['City']].value_counts())  # To check count per city
df_combined.head(2)

City     
Delhi        1485
Hyderabad    1483
Bangalore    1481
Chennai      1419
Kolkata      1381
Jaipur       1120
Name: count, dtype: int64


,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links,City
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...,Bangalore
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...,Bangalore


In [43]:
df=df_combined.copy()

In [44]:
def parse_detail(row):
    try:
        row = row.strip('"').replace("'", '"').replace("None", "null")
        return json.loads(row)
    except:
        return {}

# Apply to entire column
parsed_details = df['new_car_detail'].apply(parse_detail)

# Flatten all dicts
df_1 = pd.json_normalize(parsed_details)

# View output
print(df_1.shape)
df_1.head(2)

(8369, 19)


,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,3979,VXI,₹ 4 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,6087,1.5 Petrol Titanium BSIV,₹ 8.11 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


In [45]:
def parse_overview(row):
    try:
        row = row.strip('"').replace("'", '"').replace("None", "null")
        data = json.loads(row)
        return {item['key']: item['value'] for item in data['top']}
    except:
        return {}

# Apply to all rows
parsed = df['new_car_overview'].apply(parse_overview)

# Convert list of dicts to a DataFrame
df_2 = pd.DataFrame(parsed.tolist())

# Final result
df_2.head(2)

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2015,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0
1,Feb 2018,Comprehensive,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0


In [46]:
import pandas as pd
import json

def extract_features(row):
    try:
        # Convert string to proper JSON
        row = row.replace("'", '"').replace("None", "null")
        data = json.loads(row)
        
        # Extract features from 'top' and 'data'
        features = set()
        features.update([item['value'] for item in data.get('top', [])])
        for section in data.get('data', []):
            features.update([item['value'] for item in section.get('list', [])])
        
        return {feat: True for feat in features}
    except Exception as e:
        print("Error:", e)
        return {}

# Apply to each row and collect one-hot dictionaries
features_list = df['new_car_feature'].apply(extract_features)

# Create one-hot encoded feature matrix (DataFrame)
df_3 = pd.DataFrame(features_list.tolist()).fillna(False)

# Show shape and sample
print(df_3.shape)
df_3.head(2)

(8369, 173)


/tmp/ipykernel_21091/2494734597.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_3 = pd.DataFrame(features_list.tolist()).fillna(False)


,Side Impact Beams,Centeral Locking,Remote Fuel Lid Opener,Door Ajar Warning,Engine Immobilizer,Power Antenna,Rear Seat Belts,Adjustable Head Lights,Child Safety Locks,Anti Theft Device,...,Cassette Player,Find My Car Location,Wifi Connectivity,Headlamp Washers,Real Time Vehicle Tracking,Roof Carrier,Smart Key Band,Lane Watch Camera,Removable Convertible Top,Power Folding3rd Row Seat
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [47]:
import json
import pandas as pd

def extract_specs(row):
    try:
        # Ensure valid JSON formatting
        row = row.replace("'", '"').replace("None", "null")
        data = json.loads(row)

        specs = {}

        # Extract top-level key-value pairs
        for item in data.get('top', []):
            specs[item['key']] = item['value']

        # Extract nested 'data' specs
        for section in data.get('data', []):
            for item in section.get('list', []):
                specs[item['key']] = item['value']
        
        return specs

    except Exception as e:
        print("Error parsing row:", e)
        return {}

# Apply to entire column
specs_list = df['new_car_specs'].apply(extract_specs)

# Convert list of dicts to DataFrame
df_4 = pd.DataFrame(specs_list.tolist())

# Show shape and preview
print(df_4.shape)
df_4.head(2)


(8369, 40)


,Mileage,Engine,Max Power,Torque,Seats,Color,Engine Type,Displacement,Max Torque,No of Cylinder,...,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen
0,23.1 kmpl,998 CC,67.04bhp@6000rpm,90Nm,5,White,K10B Engine,998,90Nm@3500rpm,3.0,...,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,NaN,NaN
1,17 kmpl,1497 CC,121.31bhp@6500rpm,150Nm,5,White,Ti-VCT Petrol Engine,1497,150Nm@4500rpm,3.0,...,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,16,NaN


In [48]:
#combined dfs (including City column)
df_final=pd.concat([df[['City']],df_1,df_2,df_3,df_4],axis=1)

In [49]:
df_final.shape

(8369, 243)

In [50]:
df_final.head(2)

,City,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Front Brake Type,Rear Brake Type,Top Speed,Acceleration,Tyre Type,No Door Numbers,Cargo Volumn,Wheel Size,Alloy Wheel Size,Ground Clearance Unladen
0,Bangalore,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,...,Ventilated Disc,Drum,150 Kmph,15.05 Seconds,"Tubeless, Radial",5,235-litres,NaN,NaN,NaN
1,Bangalore,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,...,Ventilated Disc,Drum,NaN,NaN,"Tubeless,Radial",4,352-litres,16,16,NaN


In [51]:
df_final.to_csv('./parsed_data/all_cars_parsed_data.csv',index=False)